In [89]:
import pandas as pd
import sys
import warnings
import numpy as np

# used to use python functions in src modules
%load_ext autoreload
%autoreload 2
sys.path.append("..")

# pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings("ignore")

# Viz
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PARTIE 1 : Collecte des données

Pour les exercices, le cas d'usage contient des données historiques de ventes de 45 magasins Walmart aux US. Chaque magasin comprend un certain nombre de rayons et vous devez prédire les ventes de chaque rayon pour chaque magasin.

En outre, Walmart organise plusieurs opérations promotionnelles de démarque tout au long de l'année. Ces démarques précèdent des fêtes importantes, dont les quatre plus importantes sont le Super Bowl, la fête du travail, Thanksgiving et Noël.

### **fichier train.csv**

Il s'agit des données historiques sur la période du 2010-02-05 au 2012-11-01. Ce fichier contient les champs suivants :

**Store** - le numéro du magasin<br/>
**Dept** - le numéro du département<br/>
**Date** - la semaine<br/>
**Weekly_Sales** (label) - ventes hebdomadaires pour le rayon donné dans le magasin donné<br/>
**IsHoliday** - si la semaine est une semaine de vacances spéciales<br/>

### **fichier test.csv**

Le jeu de test. Vous devez prédire les ventes (Weekly_Sales) pour chaque triplet de magasin, rayon et date dans ce fichier.

### **fichier stores.csv**

Ce fichier contient des informations anonymes sur les 45 magasins, indiquant le type et la taille du magasin.

### **fichier features.csv**

Contient des données supplémentaires relatives au magasin, au rayon et à l'activité régionale pour les dates données :

**Store** - le numéro du magasin<br/>
**Date** - la semaine<br/>
**Température** - température moyenne dans la région<br/>
**Fuel_Price** - coût du carburant dans la région<br/>
**MarkDown1-5** - données anonymes relatives aux démarques promotionnelles effectuées par Walmart. Les données MarkDown ne sont disponibles qu'après novembre 2011, et ne sont pas disponibles pour tous les magasins en permanence. Toute valeur manquante est indiquée par NA.<br/>
**IPC** - l'indice des prix à la consommation<br/>
**Unemployment** - le taux de chômage<br/>
**IsHoliday** - indique s'il s'agit d'une semaine de vacances spéciales.<br/>

Par commodité, les quatre jours fériés tombent dans les semaines suivantes de l'ensemble de données (tous les jours fériés ne figurent pas dans les données) :

**Super Bowl** : 12-fév-10, 11-fév-11, 10-fév-12, 8-fév-13<br/>
**Fête du travail** : 10-sept-10, 9-sept-11, 7-sept-12, 6-sept-13<br/>
**Thanksgiving** : 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13<br/>
**Noël** : 31-déc-10, 30-déc-11, 28-déc-12, 27-déc-13


In [67]:

df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")
df_features = pd.read_csv("../data/raw/features.csv")
df_stores = pd.read_csv("../data/raw/stores.csv")

print("nombre de lignes dans le train : ", df_train.shape[0])
print("nombre de lignes dans le test : ", df_test.shape[0])


nombre de lignes dans le train :  421570
nombre de lignes dans le test :  115064


In [68]:
df_train.head(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [69]:
df_features.head(5)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [70]:
df_stores.head(5)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [71]:
df_train = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"])
df_train = df_train.merge(df_stores, on=["Store"])

df_test = df_test.merge(df_features, on=["Store", "Date", "IsHoliday"])
df_test = df_test.merge(df_stores, on=["Store"])

df_train.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,2,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,1,3,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
5,1,6,2010-02-05,5749.03,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
6,1,7,2010-02-05,21084.08,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
7,1,8,2010-02-05,40129.01,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
8,1,9,2010-02-05,16930.99,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
9,1,10,2010-02-05,30721.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315


# PARTIE 2 : Exploration des données

### Description des données et des valeurs manquantes

Les valeurs manquantes sont présentes unniquement dans les les colonnes Markdown.

In [72]:
# Adding some basic datetime features
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['Day'] = df_train['Date'].dt.day
df_train['Week'] = df_train['Date'].dt.isocalendar().week.astype(int)
df_train['Month'] = df_train['Date'].dt.month
df_train['Year'] = df_train['Date'].dt.year
df_train = df_train.drop(columns=['Date'])

df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['Day'] = df_test['Date'].dt.day
df_test['Week'] = df_test['Date'].dt.isocalendar().week.astype(int)
df_test['Month'] = df_test['Date'].dt.month
df_test['Year'] = df_test['Date'].dt.year
df_test = df_test.drop(columns=['Date'])

In [90]:
df_describe = df_train.describe().T
df_describe["% NAN"] = np.round(100 * (df_train.shape[0] - df_describe["count"]) / df_train.shape[0], 1)
df_describe.style.bar(subset=['mean'], color='#205ff2')\
                            .set_caption("Stats Summary of Numeric Variables")\
                            .background_gradient(subset=['min'], cmap='Reds')\
                            .background_gradient(subset=['max'], cmap='Greens')\
                            .background_gradient(subset=['std'], cmap='GnBu')\
                            .background_gradient(subset=['50%'], cmap='GnBu')


,count,mean,std,min,25%,50%,75%,max,% NAN
Store,421570.000000,22.200546,12.785297,1.000000,11.000000,22.000000,33.000000,45.000000,0.000000
Dept,421570.000000,44.260317,30.492054,1.000000,18.000000,37.000000,74.000000,99.000000,0.000000
Weekly_Sales,421570.000000,15981.258123,22711.183519,-4988.940000,2079.650000,7612.030000,20205.852500,693099.360000,0.000000
Temperature,421570.000000,60.090059,18.447931,-2.060000,46.680000,62.090000,74.280000,100.140000,0.000000
Fuel_Price,421570.000000,3.361027,0.458515,2.472000,2.933000,3.452000,3.738000,4.468000,0.000000
MarkDown1,150681.000000,7246.420196,8291.221345,0.270000,2240.270000,5347.450000,9210.900000,88646.760000,64.300000
MarkDown2,111248.000000,3334.628621,9475.357325,-265.760000,41.600000,192.000000,1926.940000,104519.540000,73.600000
MarkDown3,137091.000000,1439.421384,9623.078290,-29.100000,5.080000,24.600000,103.990000,141630.610000,67.500000
MarkDown4,134967.000000,3383.168256,6292.384031,0.220000,504.220000,1481.310000,3595.040000,67474.850000,68.000000
MarkDown5,151432.000000,4628.975079,5962.887455,135.160000,1878.440000,3359.450000,5563.800000,108519.280000,64.100000


### Evolution du chiffre d'affaires sur l'année

Le chiffre d'affaires hebdomadaire est très dépendant de la période de l'année et particulièrement des spéciauax (Superbowl, Noël etc...)

In [58]:
df_weeks = df_train.groupby(["Year", 'Week'])["Weekly_Sales"].sum().reset_index()

template = dict(layout=go.Layout(font=dict(family="Enriqueta", size=12))) # Cabin | Franklin Bold | Enriqueta
fig = px.line(data_frame=df_weeks, x="Week", y='Weekly_Sales', color='Year',
            template='simple_white', 
            labels={'Weekly_Sales' : 'Total Sales', 'x' : 'Weeks'})

fig.update_layout(
    template=template, 
    title={'text':'<b>Total des ventes par semaines</b>', 'x': 0.075},
    xaxis=dict(tickmode='linear', showline=True), 
    yaxis=dict(showline=True))

fig.add_annotation(
    x=0, y=-0.2, 
    align='left', 
    font=dict(size=12),
    textangle=0, 
    xref="paper", 
    yref="paper", 
    showarrow=False
    )


### Evolution du chiffre d'affaires par magasin

Le chiffre d'affaires hebdomadaire est très différent d'un magasin à l'autre même s'ils suivent à peu près la même tendance.

In [61]:
df_stores_first_10 = df_train[df_train["Store"] < 10].groupby(["Store", 'Week'])["Weekly_Sales"].sum().reset_index()

template = dict(layout=go.Layout(font=dict(family="Enriqueta", size=12))) # Cabin | Franklin Bold | Enriqueta
fig = px.line(data_frame=df_stores_first_10, x="Week", y='Weekly_Sales', color='Store',
            template='simple_white', 
            labels={'Weekly_Sales' : 'Total Sales', 'x' : 'Weeks'})

fig.update_layout(
    template=template, 
    title={'text':'<b>Total des ventes par semaines</b>', 'x': 0.075},
    xaxis=dict(tickmode='linear', showline=True), 
    yaxis=dict(showline=True))

fig.add_annotation(
    x=0, y=-0.2, 
    align='left', 
    font=dict(size=12),
    textangle=0, 
    xref="paper", 
    yref="paper", 
    showarrow=False
    )


### Repartition du chiffre d'affaires par département

In [97]:
df_depts = df_train.groupby('Dept')["Weekly_Sales"].mean().reset_index()

fig = px.bar(df_depts, x=df_depts.index, y=df_depts.Weekly_Sales, color=df_depts.Weekly_Sales)
             # color_continuous_scale=palletes['continuos']['green_n_blues'])

fig.update_layout(
    template=template, 
    title={'text':'<b>Moyenne des vents par départment</b>', 'x': 0.075},
    legend_title_text='<b>Sales</b>',
    yaxis=dict(showline=True))

fig.add_annotation(
    x=0, y=-0.20, 
    align='left', 
    font=dict(size=12),
    textangle=0, 
    xref="paper", 
    yref="paper", 
    showarrow=False
)